# Example Crash Report Notebook

Release Date: 

Target Engine Version (s): 

Analysis Date: 

Author: 



In [1]:
"""
Add module path to sys path -- necessary if running from jupyter notebook and not an IDE
"""

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
"""
General import statements
"""
import crash_analysis
import pandas as pd

"""
Constants
"""
cache_filename = 'april18data.csv'
zipfile_location = os.path.join(module_path, 'CrashAnalysis', 'tmp' + os.sep)
cache_location = os.path.join(module_path, 'CrashAnalysis', 'src', 'data' + os.sep, cache_filename)
zipfile_location

'C:\\Dev\\CrashAnalysis\\CrashAnalysis\\tmp\\'

In [4]:
"""
Download crashes by date range
"""
from crash_analysis.downloader import download_time_range
import datetime

now = datetime.datetime.now()
start = now + datetime.timedelta(days=-1)
end = now


download_time_range(start, end, zipfile_location)

Beginning download of crashes from within range 2017-09-11 10:58:14.871912 - 2017-09-12 10:58:14.871912.
About to download 239 records.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Dev\\CrashAnalysis\\CrashAnalysis\\tmp\\3b54d8cb-df5c-4de2-ba2e-497838ddaf10.zip'

In [3]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [5]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = True


if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

In [4]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)


print(col_names)
print()
print(versions)
print()
print('Number of crashes: {0}'.format(num_crashes))

['ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin15.INI', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'InvParamExpression', 'InvParamFile', 'InvParamFunction', 'InvParamLine', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin15.INI', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'TrustedCustomerException.txt', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml', 'empty']

2016100028     8520
2016120014     132

In [12]:
from crash_analysis import filter_dataframe

march_release_df = filter_dataframe(df, AppVersion=2016120014)
len(march_release_df)

814

In [14]:
march_release_df.groupby(['AppName', 'InstallType', 'OperatingSystem'])['CrashGUID'].count()

AppName                         InstallType  OperatingSystem                                          
ProSeries - 2016                Network      Windows 10 Enterprise N Build 14393                            1
                                             Windows 10 Home Build 14393                                   39
                                             Windows 10 Pro Build 10240                                     1
                                             Windows 10 Pro Build 10586                                     1
                                             Windows 10 Pro Build 14393                                    64
                                             Windows 10 Pro Build 15063                                     1
                                             Windows 7 Enterprise Build 7601                                1
                                             Windows 7 Home Premium Build 7601                             11
                 

In [24]:
def get_last_in_path(path_string):
    return str(path_string).split('\\')[-1]

march_release_df['ExceptionModule'].apply(get_last_in_path).value_counts()

nan                                    213
protax16.exe                           148
KERNELBASE.dll                         135
AcroPDFImpl.dll                         90
ntdll.dll                               69
c4dll.dll                               69
CrashRpt1403.dll                        28
user32.dll                              15
TerR.dll                                11
mfc140.dll                               6
ucrtbase.dll                             5
Rep32.dll                                5
ieframe.dll                              4
shell32.dll                              2
PSTPRX32.DLL                             2
mso20win32client.dll                     2
Flash32_25_0_0_148.ocx                   1
ole32.dll                                1
mshtml.dll                               1
PSLogR.DLL                               1
msxml6.dll                               1
kernel32.dll                             1
tv_w32.dll                               1
QBInstanceF

In [27]:
march_release_df[march_release_df['ExceptionAddress'] == '0x100427a3']['CustNum'].value_counts()

0000216971    2
0006404461    2
0297743451    2
7755643535    1
0478204787    1
0574525101    1
0032650866    1
7700765181    1
0768698052    1
7701116789    1
7771106389    1
0829781505    1
7770424368    1
0000178935    1
0606910107    1
0259128155    1
0004373593    1
0606907178    1
0276439783    1
0260458765    1
0000550391    1
0357795244    1
0111723507    1
0000999641    1
0755622718    1
0607152303    1
0271699993    1
0295164644    1
0310013321    1
0606995991    1
7702384869    1
7702764938    1
0000957761    1
7770838272    1
Name: CustNum, dtype: int64